# ERA5 & MERIT: Monthly Potential Runoff 

*Definition* 
Potential Runoff: Sum of computed run off in the contributing area of a point. 

Taking a super simple approach because we are interested in
relatively small areas. The simplifying assumption is that the runoff upstream of a point is the same as at the point. This would clearly not be a reasonable assumption for large areas. It may also not be a good assumption for arid areas. The full runoff integration at a point would require topological tracing, which appears a bit akward in EE and quite a bit more work. 

Going with the simplifying assumptions, take ERA5 surface runoff (meters) and multiply it by MERIT upstream area (km^2, converted to m^2)

(This could likewise be done for subsurface runoff, even more sketchy!)

This notebook constructs the monthly climatologies but does not export of visualize them.

In [1]:
import ee

import sys
sys.path.insert(1, '../SMAP-USDA/')  # until we package this
import mon_stats

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
# ERA5 Monthly Average (of hourly) Surface runoff 
# https://www.ecmwf.int/sites/default/files/elibrary/2016/17117-part-iv-physical-processes.pdf#subsection.H.6.3
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY#bands
era5_month_sfc_runoff_m = (
    ee
    .ImageCollection("ECMWF/ERA5_LAND/MONTHLY")
    .select('surface_runoff'))

In [5]:
era5_sfc_runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], era5_month_sfc_runoff_m)

In [6]:
era5_sfc_runoff_clim_m['avgs'].getInfo()

{'surface_runoff': {'type': 'ImageCollection', 'bands': []}}

In [7]:
era5_sfc_runoff_clim_m['lens'].getInfo()

{'surface_runoff': [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 39]}

In [8]:
# MERIT Upstream Area 
# https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description
merit_drain_area_m2 = (
    ee
    .Image("MERIT/Hydro/v1_0_1")
    .select('upa')
    .multiply(1e6))  ## km2 -> m2

In [9]:
# Potential Runoff - combine and put in average m^3/s
# m3/hr = m3 / (60min*60s)
def runoff_times_contrib_area_cms(month_key, month):
    runoff_m = (
        ee.ImageCollection(
            era5_sfc_runoff_clim_m['avgs'].get('surface_runoff'))
        .filter(ee.Filter.eq('month', month)).first())
    return (
        merit_drain_area_m2
        .multiply(runoff_m)
        .multiply(1.0/(60*60))
        .set('month', month).set('variable', 'potential_sfc_runoff_mon_clim_cms'))

In [10]:
months_dict = mon_stats.months_dict()

In [11]:
potential_sfc_runoff_clim_cms = months_dict.map(runoff_times_contrib_area_cms)
# potential_sfc_runoff_clim_cms.getInfo()

In [12]:
 potential_sfc_runoff_clim_cms.getInfo()

{'01_Jan': {'type': 'Image',
  'bands': [{'id': 'upa',
    'data_type': {'type': 'PixelType', 'precision': 'double'},
    'crs': 'EPSG:4326',
    'crs_transform': [0.0008333333333333334,
     0,
     -180.00041666666667,
     0,
     -0.0008333333333333334,
     84.99958333333333]}],
  'properties': {'month': 1, 'variable': 'potential_sfc_runoff_mon_clim_cms'}},
 '02_Feb': {'type': 'Image',
  'bands': [{'id': 'upa',
    'data_type': {'type': 'PixelType', 'precision': 'double'},
    'crs': 'EPSG:4326',
    'crs_transform': [0.0008333333333333334,
     0,
     -180.00041666666667,
     0,
     -0.0008333333333333334,
     84.99958333333333]}],
  'properties': {'month': 2, 'variable': 'potential_sfc_runoff_mon_clim_cms'}},
 '03_Mar': {'type': 'Image',
  'bands': [{'id': 'upa',
    'data_type': {'type': 'PixelType', 'precision': 'double'},
    'crs': 'EPSG:4326',
    'crs_transform': [0.0008333333333333334,
     0,
     -180.00041666666667,
     0,
     -0.0008333333333333334,
     84.9995